In [2]:
pip install git+https://github.com/jkirkby3/pymle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/jkirkby3/pymle to /tmp/pip-req-build-6fp5ti7_
  Running command git clone -q https://github.com/jkirkby3/pymle /tmp/pip-req-build-6fp5ti7_
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for PyMLE: filename=PyMLE-0.0.1-py3-none-any.whl size=38015 sha256=e5854baca4a6dfaa80f067ecfeca3dbe96231309c80c5ba2f391436c6b625e73
  Stored in directory: /tmp/pip-ephem-wheel-cache-9l0w4stn/wheels/cd/1e/47/b1240ec565910918e972d8bc400bc27859de0658a5cc94b937
Successfully built PyMLE


# **1) Генерация OU и оценка его параметров**

In [3]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random

from scipy.stats import norm

from pymle.models import OrnsteinUhlenbeck
from pymle.sim.Simulator1D import Simulator1D
from pymle.TransitionDensity import ExactDensity
from pymle.TransitionDensity import EulerDensity
from pymle.fit.AnalyticalMLE import AnalyticalMLE


In [4]:
def OU(X, kappa, mu, sigma, dW, dt):
  dX = kappa * (mu - X) * dt + sigma * dW
  return dX

In [11]:
def beta1(X, n):
  sum1, sum2, sum3, sum4 = 0, 0, 0, 0
  for i in range(1, n):
    sum1 += X[i] * X[i - 1]
    sum2 += X[i]
    sum3 += X[i - 1]
    sum4 += X[i - 1]**2
  bet1 = (sum1 / n - sum2 * sum3 / n**2) / (sum4 / n - sum3**2 / n**2)
  return bet1


def beta2(X, n, b1):
  sum = 0
  for i in range (1, n):
    sum += (X[i] - b1 * X[i - 1])
  res = (sum / n) / (1 - b1)
  return res

def beta3 (X, n, b1, b2):
  sum = 0
  for i in range (1, n):
    sum += (X[i] - b1 * X [i - 1] - b2 * (1 - b1))** 2
  res = sum / (n * b1)
  return res

In [12]:
kappa = 3
mu = 1
sigma = 2
N = 1000
delta = 1 / N
dW = np.random.normal(0, np.sqrt(1 / N), N)

X_ou = np.zeros(N)
X_ou[0] = 10
for i in range(1, N):
    dX = OU(X_ou[i - 1], kappa, mu, sigma, dW[i], delta)
    X_ou[i] = X_ou[i - 1] + dX

In [13]:
b_1 = beta1(X_ou, len(X_ou))
b_2 = beta2(X_ou, len(X_ou), b_1)
b_3 = beta3(X_ou, len(X_ou), b_1, b_2)

kappa_est= (-1 / delta) * np.log(b_1)
mu_est = b_2
sigma_est = (2 * kappa_est * b_3 / (1 - b_1 ** 2))**0.5

print('kappa_estimation =', kappa_est)
print('mu_estimation =', mu_est)
print('sigma_estimation =', sigma_est)

kappa_estimation = 3.217928703122882
mu_estimation = 0.9220942927849521
sigma_estimation = 1.9712367947016276


Через библиотекку

In [23]:

model = OrnsteinUhlenbeck()  

kappa = 3  # rate of mean reversion
mu = 1  # long term level of process
sigma = 2  # volatility

model.params = np.array([kappa, mu, sigma])


S0 = 10  # initial value of process
T = 1000  # number of days of the sample
freq = 1  # observations per day
dt = 1. / freq
seed = 123  # random seed: set to None to get new results each time

#simulator = Simulator1D(S0=S0, M=T * freq, dt=dt, model=model).set_seed(seed=seed)
#sample = simulator.sim_path()



param_bounds = [(0, 6), (0, 6), (0, 6)]

guess = np.array([1, 0.1, 1])

exact_est = AnalyticalMLE(X_ou, param_bounds, dt, density=ExactDensity(model)).estimate_params(guess)

print(f'\nExact MLE: {exact_est}')

Initial Params: [1.  0.1 1. ]
Initial Likelihood: -9556.177848045578
`xtol` termination condition is satisfied.
Number of iterations: 72, function evaluations: 264, CG iterations: 138, optimality: 1.28e-05, constraint violation: 0.00e+00, execution time: 0.28 s.
Final Params: [0.00299477 0.92357542 0.06136852]
Final Likelihood: 1372.042837996194

Exact MLE: 
params      | [0.00299477 0.92357542 0.06136852] 
sample size | 999 
likelihood  | 1372.042837996194 
AIC         | -2738.085675992388
BIC         | -2723.3654116564426


/usr/local/lib/python3.8/dist-packages/scipy/optimize/_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '


# **Задача 2) Явная оценка парметров и оценка методом эйлера**

In [14]:
kappa = 0
alpha = 3
sigma = 2
N = 1000
delta = 1 / N
dW = np.random.normal(0, np.sqrt(1 / N), N)

In [18]:
def theta_2(X, delta, n):
  t1 = 0
  t2 = 0
  for i in range(1, n):
    t1 += X[i] * X[i - 1]
    t2 += X[i-1] ** 2
  t_2 = -np.log(t1 / t2) / delta
  return t_2

def theta_3(X, delta, n, t_2):
  t1 = 0
  t2 = 0
  for i in range(1, n):
    t1 += (X[i] - X[i - 1] * np.exp(-delta * t_2)) ** 2 
  t2 = (2 * t_2) / (n * (1 - np.exp(-2 * delta * t_2)))
  return t1 * t2


In [21]:
X_ou = np.zeros(N)
X_ou[0] = 10
for i in range(1, N):
    dX = OU(X_ou[i - 1], 3, 0, 2, dW[i], 1 / N)
    X_ou[i] = X_ou[i - 1] + dX

In [22]:
theta2_est = theta_2(X_ou, delta, len(X_ou))
theta3_est = theta_3(X_ou, delta, len(X_ou), theta2_est) ** 0.5


print('Theta_2=', theta2_est)
print('Theta_3=', theta3_est)

Theta_2= 2.7727369823533032
Theta_3= 2.030540077710983


Через библиотеку:

In [35]:
model = OrnsteinUhlenbeck()  
model.params = np.array([kappa, mu, sigma])

# ===========================
# Simulate a sample path (we will fit to this path)
# ===========================
S0 = 10  # initial value of process
T = 1000  # num years of the sample
freq = 1  # observations per year
dt = 1. / freq
seed = 123  # random seed: set to None to get new results each time

X = X_ou


X = np.array(X).reshape(-1)
# ===========================
# Fit maximum Likelihood estimators
# ===========================
# Set the parameter bounds for fitting  (kappa, mu, sigma)
param_bounds = [(-1, 5), (-1, 5), (-1, 5)]

# Choose some initial guess for params fit
guess = np.array([0.5, 0.1, 0.4])

# Fit using Exact MLE
exact_est = AnalyticalMLE(X, param_bounds, dt, density=ExactDensity(model)).estimate_params(guess)

# Fit using Euler MLE
euler_est = AnalyticalMLE(X, param_bounds, dt, density=EulerDensity(model)).estimate_params(guess)


print(f'\nExact MLE:\n', exact_est)

print(f'\nEuler MLE:', euler_est)

Initial Params: [0.5 0.1 0.4]
Initial Likelihood: -13993.141847594055
`xtol` termination condition is satisfied.
Number of iterations: 82, function evaluations: 384, CG iterations: 209, optimality: 6.32e-03, constraint violation: 0.00e+00, execution time: 0.36 s.
Final Params: [0.00369219 1.55508991 0.06265772]
Final Likelihood: 1351.62178702136
Initial Params: [0.5 0.1 0.4]
Initial Likelihood: -14529.904361718482
`xtol` termination condition is satisfied.
Number of iterations: 70, function evaluations: 328, CG iterations: 170, optimality: 1.21e-03, constraint violation: 0.00e+00, execution time: 0.26 s.
Final Params: [0.0036853  1.55498825 0.06254223]
Final Likelihood: 1351.6217869906354

Exact MLE:
 
params      | [0.00369219 1.55508991 0.06265772] 
sample size | 999 
likelihood  | 1351.62178702136 
AIC         | -2697.24357404272
BIC         | -2682.5233097067744

Euler MLE: 
params      | [0.0036853  1.55498825 0.06254223] 
sample size | 999 
likelihood  | 1351.6217869906354 
AIC  

# **Задача 3)**

In [39]:
from pymle.models import CKLS
import scipy.stats as st


model = CKLS()  
model.params = np.array([1, 2, 0.5, 0.3])

S0 = 2  # initial value of process
T = 2  # num years of the sample
freq = 1000  # observations per year
dt = 1. / freq
seed = None#123  # random seed: set to None to get new results each time

simulator = Simulator1D(S0=S0, M=T * freq, dt=dt, model=model).set_seed(seed=seed)
sample = simulator.sim_path()

param_bounds = [(0, 5), (0, 5), (0, 5), (0, 5)]

guess = np.array([0.5, 0.9, 0.9, 0.9])

theta1 = []
theta2 = []
theta3 = []
theta4 = []

for i in range (0, 100):
    print(i)
    simulator = Simulator1D(S0=S0, M=T * freq, dt=dt, model=model).set_seed(seed=seed)
    sample = simulator.sim_path()
    euler_est = AnalyticalMLE(sample, param_bounds, dt, density=EulerDensity(model)).estimate_params(guess)
    theta1.append(euler_est.params[0])
    theta2.append(euler_est.params[1])
    theta3.append(euler_est.params[2])
    theta4.append(euler_est.params[3])

print("Theta_1: ", st.t.interval(alpha=0.95, df=len(theta1)-1, loc=np.mean(theta1), scale=st.sem(theta1)))

print("Theta_2: ", st.t.interval(alpha=0.95, df=len(theta2)-1, loc=np.mean(theta2), scale=st.sem(theta2)) )

print("Theta_3: ", st.t.interval(alpha=0.95, df=len(theta3)-1, loc=np.mean(theta3), scale=st.sem(theta3)))

print("Theta_4: ", st.t.interval(alpha=0.95, df=len(theta4)-1, loc=np.mean(theta4), scale=st.sem(theta4)))

print(f'\nEuler MLE: {euler_est} ')

0
Initial Params: [0.5 0.9 0.9 0.9]
Initial Likelihood: 160.02364743459714
`xtol` termination condition is satisfied.
Number of iterations: 69, function evaluations: 325, CG iterations: 202, optimality: 1.18e-04, constraint violation: 0.00e+00, execution time: 0.34 s.
Final Params: [0.33853504 2.04491069 0.51187193 0.28481604]
Final Likelihood: 3797.1728816640857
1
Initial Params: [0.5 0.9 0.9 0.9]
Initial Likelihood: 164.05280359245307
`xtol` termination condition is satisfied.
Number of iterations: 53, function evaluations: 210, CG iterations: 154, optimality: 1.24e-05, constraint violation: 0.00e+00, execution time: 0.23 s.
Final Params: [0.70586542 2.0357304  0.51586934 0.28830075]
Final Likelihood: 3763.9406642620143
2
Initial Params: [0.5 0.9 0.9 0.9]
Initial Likelihood: 37.3926400731915
`xtol` termination condition is satisfied.
Number of iterations: 65, function evaluations: 315, CG iterations: 173, optimality: 1.05e-04, constraint violation: 0.00e+00, execution time: 0.32 s.
F

In [45]:
print(f"Theta_1 = {np.array(theta1).mean()}, Theta_2 = {np.array(theta2).mean()}")
print(f"Theta_3 = {np.array(theta3).mean()}, Theta_4 = {np.array(theta4).mean()}")

Theta_1 = 1.3068204876865273, Theta_2 = 2.1585841212358012
Theta_3 = 0.4118806749994789, Theta_4 = 0.3806431200042448
